In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
# from einops import rearrange, repeat
# import rosbags
import json
import open3d as o3d
import pandas as pd
import os
import copy
from utility import listFiles, quickVizNumpy
from scipy.spatial.transform import Rotation as R
from numpy import cos, sin, sqrt, arctan2, array
import torch.nn as nn
### fucking around
from pretrain import loadDataFromRegisteredSlam, cart2sph, preProcess
from train import getSamples, getUpSamples, getSpacing
from visualize import visualizeDir


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
name = r'building_with_rotation'
path = r'datasets/registered/' + name + r'.json'
data = loadDataFromRegisteredSlam(path)
# training_data = preProcess(data)
# np.random.shuffle(training_data)

In [69]:
path = r'/home/ansonhon/anson/thesis/LiDAR_NeRF/datasets/training_euclidean/building.npy'

data_euclidean = np.load(path)

In [80]:
laser_org = data_euclidean[:,0:3]
laser_dir = data_euclidean[:,3:6]
laser_pts = data_euclidean[:,6:9]

In [84]:
temp = laser_pts[400000:600000,:]
quickVizNumpy(temp)